# Filtering prescriptions
First, we find the number of observation that has each unit of measurement. If the top frequent one has the observations less than 90% of total, then we discard it.

Also, there are many medicine that never be used by any patients, so we also discard it.

In [1]:
# DB connection:
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
%matplotlib inline

# connect DB

In [2]:
try:
    conn = psycopg2.connect("dbname='mimic' user='mimic' host='localhost' password='meladymimic315'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


# load admissionID and itemID

In [3]:
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

db = np.load('res/itemids.npy').tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']
print len(prescript_itemid)

3268


# Let's explain it later (A)

In [4]:
manual_valid = ['CALG1I','CEFA10I','CEFX1F','ZOSY4.5I','CEFX2F','UNAS3I','OCTR0.1I','MANN100I','MANN20PM','GAMML10I','AMP1I','ERTA1I','PHEN50I','PENGK20I','PENGK3I','EPIBAG2','TALC5','NAFC1F','AMIN500','CEFA1I','ALDE22I','PROB500','CEFR1I','NAFC2I','LIEP550I','PENI2F','FOSF3','MESA4R','GAMU5I','PENGK5I','PCNGB240I','CEFR2I','CEF1000I','LCARN250','MURO5O','CIME150I','ALBELI120']

# filterout if the unit is not consistent, using the same method as other events.

### This one is a little different, there are many cases that accidentally discard. For example this one.

CALG1I : 70.33 : 3 : [('gm', 5522L), ('g', 2322L), ('', 7L)]

### For this one we can translate gm to g, but the program see gm different from g. Actually I should group the unit of measurement that we can translate it with each other first. 

### I saw that there are not much, so I manually select them so I got list A above. The list A here is valid, we never discard it !

In [13]:
valid_prescript =[]
valid_prescript_unit =[]
dropped_id = []
notfound = []
# prescriptions
ct = 0
for i in prescript_itemid:
    print ct,
    ct+=1

    # foreach medicine, list the dose unit of medicine and the count of observations that use that unit.
    cur = conn.cursor()
    cur.execute('SELECT dose_unit_rx, count(dose_unit_rx) FROM mimiciii.prescriptions WHERE formulary_drug_cd = \''+ str(i) +'\' and hadm_id in ('+admission_ids_txt+') group by dose_unit_rx')
    outputunits = cur.fetchall()
    
    # sort it descendently
    outputunits = sorted(outputunits, key=lambda tup: tup[1])
    outputunits.reverse()
    
    # check if medicine is never used by anybody, then discard it.
    total = 0
    for o in outputunits:
        total += o[1]
    if(total == 0 ): 
        print "\n**PRES DROPPED "+str(i)
        notfound.append(i)
        for o in outputunits:
            print o
        print ''
        continue;
        
    # calculate the percentage of observation of main unit.
    percentage = float(outputunits[0][1]) / total *100.
    if(percentage < 90): 
        # never drop the list A
        if(i in manual_valid):
            print "\n\n****PRES NOT DROPPED "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)+"\n"
        
        # drop it !
        else:
            dropped_id.append(i)
            print "\n\n****PRES DROPPED "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)+"\n"
            continue;
#     print "PRES "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)
    
    # keep it and also save the main unit of it.
    valid_prescript.append(i);
    valid_prescript_unit.append(outputunits[0][0]);

 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 

****PRES NOT DROPPED CALG1I : 70.33 : 3 : [('gm', 5522L), ('g', 2322L), ('', 7L)]

91 92 93 94 95 96 97 98 99 

****PRES NOT DROPPED CEFA10I : 87.13 : 3 : [('g', 3901L), ('gm', 568L), ('mg', 8L)]

100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 

****PRES NOT DROPPED CEFX1F : 87.48 : 3 : [('gm', 2844L), ('g', 404L), ('mg', 3L)]

117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 

****PRES NOT DROPPED ZOSY4.5I : 67.34 : 3 : [('g', 1534L), ('gm', 742L), ('mg', 2L)]

153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 

****PRES DROPPED ACET20/4I : 69.43 : 2 : [('mg', 1644L), ('mL', 724L)]

170 171 1

In [16]:
print 'len(prescript_itemid) = ' + str(len(prescript_itemid))
print 'len(valid_prescript) = ' + str(len(valid_prescript))
print 'len(dropped_id) = ' + str(len(dropped_id))
print 'len(notfound) = ' + str(len(notfound))

len(prescript_itemid) = 3268
len(valid_prescript) = 2371
len(dropped_id) = 85
len(notfound) = 812


# save it !

In [11]:
np.save('res/filtered_prescript',{'id':valid_prescript,'unit':valid_prescript_unit});
print "saved !";

saved !
